In [2]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


/Users/trancatkhanh/Downloads/GAME


# Extra Homework 1:

*  **1.For each action , save at least 200k state both old and new state** 
*  **2.Label the state  which change 0 and 1** 
*  **3.Predict what state will get reward** 

**FILE_DATA_CỦA_TỪNG_ACTION:** 


https://drive.google.com/drive/folders/10clf3QG6yf3Epdj5Z-MpWjkPs_gQad-Q?usp=sharing


**1.For each action , save at least 200k state both old and new state** 

In [ ]:
import numpy as np
from base.Splendor_v3.env import *
for k in range(2,15):
  @njit()
  def p0(state,per):
    # Choose an action
    actions = getValidActions(state)
    actions = np.where(actions == 1)[0]
    action = actions[np.random.randint(len(actions))]
    # Lưu new state
    if  per[3][0][0] == 1: 
      if int(per[4][0][0])<=200000:
        j = int(per[4][0][0])
        for i in range(getStateSize()):
          per[2][j][i] += state[i]
        # print('lưu state sau khi action '+f'{k}',per[2][j])
        # trigger action 2
        per[3][0][0] = 0
        per[4][0][0] += 1
        # print("action sau khi action "+f'{k}',action)

    #Lưu old state 
    if action == k :
      if int(per[1][0][0])<=200000:
        # save state
        j = int(per[1][0][0])
        for i in range(getStateSize()):
          per[0][j][i] += state[i]
        # print('lưu state trước khi action '+f'{k}',per[0][j])
        # count number of state
        per[1][0][0] += 1
        per[3][0][0] = 1
        # print("action ",action)
      
    return action,per

          #0                                  #1                #2                              #3            #4
  perx = [np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((1,1))]
            #state old                      #số trận của old       #state new                  # trigger      # số trận của new
  win, x = numba_main_2(p0, 200000,perx , 0)
  np.save('old'+f'{k}'+'.npy',x[0])
  np.save('new'+f'{k}'+'.npy',x[2])

**2.Label the state  which change 0 and 1** 


In [ ]:
old = np.load('old0.npy')
new = np.load('new0.npy')
#Label the data (the state will give reward : 1) and (the state will not give reward : 0)
o = (new>old)
o= 1*o
i = np.zeros((245142,1))
for u in range(245142) :
    if sum(o[u])>0 :
        # print(o[u])
        i[u] = 1

**3.Predict what value in state will change** 


**======> Apply Logistic regression**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_img, test_img, train_lbl, test_lbl = train_test_split(old, i, test_size=1/7.0, random_state=0)

clf1 = LogisticRegression(random_state=0).fit(train_img, train_lbl)
clf1.score(test_img, test_lbl)


0.9730130655559993

In [ ]:
print(np.where(i==0)[0])
print(old[30])
print(new[30])
# Predict the state will give reward or not
print(clf1.predict(old[52].reshape(1,161)))

[0.]


# Bài tập 1 : Tìm ra feature hoặc bộ feature cho từng action để tỉ lệ +value khi active = 1 


In [3]:
def get_feature(i):
  #State old là state Hiện tại 
  #State new là state Mới 
  old = np.load('old'+f'{i}'+'.npy')
  new = np.load('new'+f'{i}'+'.npy')
  rate = 0 
  while rate<0.999 :

    # state mà có cả Value (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  create a feature_matrix with random value
    l = np.random.uniform(-1,1,(161,1))
    for j in range(getStateSize()):
        if l[j] > np.random.randn(1):
            l[j] = 1
        elif l[j] < np.random.randn(1):
            l[j] =  0
        else:
            l[j] = -1
    #state mà kích hoạt (1,0)
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
    rate = (sum(y)/sum(k))
  return l.reshape(1,161)



# Bài tập 2 : Khi dùng bộ feature đó cho từng action, tỉ lệ thắng >35% 


**Check win rate of each feature of each action**
==> In ra số trận thắng của từng action và lưu lại nó trong file per


In [7]:
# Check win rate of each feature must > 35%
matrix_feature_check_win_rate = []

for k in range(0,15): 
  matrix  = get_feature(k)
  matrix_feature = []
  matrix_feature.append(matrix)
  perx=[np.zeros((len(matrix_feature),getStateSize()))]
  for i in range(0,len(matrix_feature)):
      for j in range(getStateSize()):
          perx[0][i][j] += matrix_feature[i][0][j]
  @njit
  def p0(state,per):
      actions = getValidActions(state)
      matran1 = per[0][0]@state    
      matran1 = int(matran1 >= 0)
      actions = np.where(actions == 1)[0]
      if matran1 == 1 and k in actions :
          action = k
      else:
          action = actions[np.random.randint(len(actions))]
      return action,per
  win, x = numba_main_2(p0, 1000,perx,0)
  while  win<=350 :
    matrix  = get_feature(k)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx=[np.zeros((len(matrix_feature),getStateSize()))]
    for i in range(0,len(matrix_feature)):
        for j in range(getStateSize()):
            perx[0][i][j] += matrix_feature[i][0][j]
    win, x = numba_main_2(p0, 1000,perx,0)
  print('số trận thắng của action '+f'{k}',win)
  matrix_feature_check_win_rate.append(matrix)


số trận thắng của action 0 357
số trận thắng của action 1 455
số trận thắng của action 2 523
số trận thắng của action 3 564


**Lưu lại các feature**

In [ ]:
# create weight for each feature
perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75))]
for vk in range(len(matrix_feature_check_win_rate)):
  for i in range(0,5):
      for j in range(getStateSize()):
          perx[0][vk*5+i][j] += matrix_feature_check_win_rate[vk][i][j]
for i in range(0,15):
    for j in range(5):
        perx[1][i][i*5+j] = 1

IndexError: index 1 is out of bounds for axis 0 with size 1

# <Phần mở rộng> 
# Ghép các action với nhau 

**Bước 1 :Lấy bias của các action**

In [51]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

[0.93169458 0.95408836 0.96489867 1.         0.88563347 0.90320685
 0.90613141 0.90128938 0.9156278  0.90590432 0.91849948 0.90841397
 0.91312464 0.89787697 0.91588065]
555


In [71]:
perx=[np.zeros((len(matrix_feature),getStateSize()))]
for i in range(0,len(matrix_feature)):
    for j in range(getStateSize()):
        perx[0][i][j] += matrix_feature[i][0][j]
print(perx)

[array([[ 1., -1., -1.,  1.,  0.,  1.,  0., -1.,  0.,  1., -1.,  1.,  1.,
         0.,  0., -1., -1.,  0.,  1.,  0.,  1.,  1.,  1.,  1., -1.,  1.,
        -1.,  1.,  1.,  0.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        -1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  0., -1.,  1.,
         1.,  0.,  1.,  0.,  0., -1.,  1.,  1., -1.,  0.,  0.,  0.,  1.,
         1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  1.,
        -1.,  1.,  1.,  0., -1.,  1.,  1.,  1.,  0.,  0., -1.,  1.,  1.,
        -1., -1., -1.,  0., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,
         1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1., -1.,
         0.,  0.,  0., -1.,  1.,  0.,  1., -1.,  1., -1.,  0.,  1.,  0.,
         1.,  0.,  1.,  0.,  0.,  1.,  1.,  1., -1.,  1., -1.,  0., -1.,
         1.,  0.,  0.,  1.,  0.],
       [ 1.,  1., -1.,  1.,  1., -1.,  0.,  0.,  0., -1.,  1.,  0., -1.,
         1.,  1.

# Cách 1 

In [77]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state       
    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1[1] == 1 and matran1[0] == 0:
        action = 1
        print(matran1)

    elif matran1[0] == 1 and matran1[1] == 0 :
        action = 0
        print(matran1)

    elif matran1[0] == matran1[1] and matran1[0] == 1:
        if bias_greedy[0]+1 > bias_greedy[1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 10000,perx,0)
print('tỷ lệ thắng',(win/10000)*100)

[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
tỷ lệ thắng 45.95


#  Cách 2: 

In [78]:
perx1 =[bias_greedy]

In [ ]:
l = (matrix_feature[0][0])
print(l)
l1 = (matrix_feature[1][0])
print(l1)

In [109]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = l@state   
    matran2 = l1@state   
    matran1 = int(matran1 >= 0)
    matran2 = int(matran2 >= 0)
    if matran1 > matran2 :
        action = 0
    elif matran2 > matran1  :
        action = 1
    elif matran2 == matran1 :
        if per[0][0]+1 > per[0][1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 10000,perx1,0)
print('tỷ lệ thắng',(win/10000)*100)

tỷ lệ thắng 46.36
